In [1]:
# The first part of this code utilizes with some modifications code by Sung Pil Moon from https://blog.nycdatascience.com/student-works/project-3-web-scraping-company-data-from-indeed-com-and-dice-com/
# load the library
from bs4 import BeautifulSoup as Soup
import urllib, requests, re, pandas as pd
import time
import re

# indeed.com url
base_url = 'http://www.indeed.com/jobs?q=data+scientist&jt=fulltime&sort='
# sort by data
sort_by = 'date'
# start page number
start_from = '&start=' 

pd.set_option('max_colwidth',500)

In [2]:
df = pd.DataFrame()

# page from 1 to 100 (last page we can scrape is 100)
for page in range(1,101): 
    time.sleep(3)
    print(page)
    page = (page-1) * 10  
    # get full url 
    url = "%s%s%s%d" % (base_url, sort_by, start_from, page) 
    target = Soup(urllib.request.urlopen(url), "lxml") 

    # we're interested in each row (= each job), combine multiple lists to deal with
    # usages of spaces
    targetElements = target.find_all('div', attrs={'class' : ' row result'})
    targetElements2 = target.find_all('div', attrs={'class' : 'row result'})
    targetElements3 = target.find_all('div', attrs={'class' : '  row result'})
    targetElements4 = target.find_all('div', attrs={'class' : '   row result'})
    targetElements = targetElements + targetElements2 + targetElements3 + targetElements4
    
    # trying to get each specific job information (such as company name, job title, urls, ...)
    for elem in targetElements:
        try:
            comp_name = elem.find('span', attrs={'class':'company'}).getText()
        except AttributeError:
            comp_name = None
        try:
            job_title = elem.find('a', attrs={'class':'turnstileLink'}).attrs['title']
        except AttributeError:
            job_title = None
        home_url = "http://www.indeed.com"
        job_link = "%s%s" % (home_url,elem.find('a').get('href'))
        job_loc = elem.find('span', attrs={'class':'location'}).getText()
        job_sum = elem.find('span', attrs={'class':'summary'}).getText()

        # add a job info to our data frame
        df = df.append({'comp_name': comp_name, 'job_title': job_title, 
                        'job_link': job_link, 'job_loc' : job_loc, "job_summary" : job_sum
                       }, ignore_index=True)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100


In [3]:
df.to_csv("job_ad_metadata_v2.csv", index = False)
print(df.shape)
df.head()

(994, 5)


,comp_name,job_link,job_loc,job_summary,job_title
0,\n\n\n Telus International\n,http://www.indeed.com/rc/clk?jk=77be2b9c3776bd7c&fccid=1a93caa31afcad42,"Las Vegas, NV","\nYour primary focus will be in applying data mining techniques, doing statistical analysis, and building high quality prediction systems integrated with our...",Data Scientist
1,\n\n\n Anadarko Petroleum Corporation\n,http://www.indeed.com/rc/clk?jk=a8dcfbb9a24e2bf6&fccid=7057ab93f0d7d306,"Midland, TX 79705","\nThe Data Scientist will also work with the centralized Advanced Analytics & Emerging Technology organization, including the Data Science & Advanced Analytics...",Data Scientist - Delaware Basin Analytics & Data Science
2,\n\n\n Intuit\n,http://www.indeed.com/rc/clk?jk=575b9ab4cc4b8033&fccid=9784ae78e9834539,"Mountain View, CA 94039","\nApply data mining, NLP, and machine learning (both supervised and unsupervised) to improve relevance and personalization algorithms ....",Staff Data Scientist
3,\n\n\n Intuit\n,http://www.indeed.com/rc/clk?jk=6e0d00c32933e2de&fccid=9784ae78e9834539,"Woodland Hills, CA 91367","\nGovernment and other agencies to ensure influence into emerging industry trends. Experience with statistical modeling techniques like logistic regression,...",Staff Data Scientist of Innovation
4,\n\n\n Intuit\n,http://www.indeed.com/rc/clk?jk=48082c07ae3c6d12&fccid=9784ae78e9834539,"Mountain View, CA 94039","\nApply data mining, NLP, and machine learning (both supervised and unsupervised) to improve relevance and personalization algorithms ....",Senior Data Scientist
